In [7]:
!pip install tensorflow tensorflow-hub numpy matplotlib scikit-learn


[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# Import library yang dibutuhkan
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score

In [9]:
# Konfigurasi parameter untuk model dan data
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATASET_DIR = 'batik_jogja' # Pastikan folder ini ada di direktori yang sama
LEARNING_RATE = 0.001
EPOCHS = 200 # Anda bisa menambahkannya untuk akurasi yang lebih baik

# URL model EfficientNet-Lite0 dari TensorFlow Hub
HUB_MODEL_URL = "https://tfhub.dev/tensorflow/efficientnet/lite0/feature-vector/2"

In [10]:
print("Memuat dataset dari direktori...")

# Dataset untuk training (80% dari data)
train_dataset = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="training",
    seed=123
)

# Dataset untuk validasi (20% dari data)
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="validation",
    seed=123
)

# Simpan nama kelas
class_names = np.array(train_dataset.class_names)
num_classes = len(class_names)
print("\nKelas yang ditemukan:", class_names)

Memuat dataset dari direktori...
Found 1350 files belonging to 6 classes.
Using 1080 files for training.
Found 1350 files belonging to 6 classes.
Using 270 files for validation.

Kelas yang ditemukan: ['batik_ceplok' 'batik_kawung' 'batik_nitik' 'batik_parang'
 'batik_sidoluhur' 'batik_truntum']


In [11]:
# Menggunakan cache dan prefetch untuk mempercepat loading data saat training
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

print("\nDataset siap digunakan.")


Dataset siap digunakan.


In [12]:
# Cell 5: Membangun Model (Menggunakan Model Subclassing - SOLUSI FINAL)

print("Membangun model Keras dengan metode Subclassing...")

# 1. Definisikan arsitektur model di dalam sebuah class turunan dari tf.keras.Model
class BatikLiteModel(tf.keras.Model):
  def __init__(self, num_classes):
    super().__init__()
    # Definisikan semua layer yang dibutuhkan di sini
    self.feature_extractor = hub.KerasLayer(
        HUB_MODEL_URL,
        trainable=False,
        name='efficientnet_lite0_base'
    )
    self.dropout_layer = tf.keras.layers.Dropout(0.2)
    self.output_layer = tf.keras.layers.Dense(num_classes, activation='softmax')

  # Definisikan alur maju (forward pass) data di dalam method 'call'
  def call(self, inputs):
    x = self.feature_extractor(inputs)
    x = self.dropout_layer(x)
    x = self.output_layer(x)
    return x

# 2. Buat sebuah instance (objek) dari class model yang baru kita buat
model = BatikLiteModel(num_classes=num_classes)

# 3. (Penting) Panggil `build` agar model tahu bentuk inputnya dan .summary() bisa ditampilkan
model.build((None,) + IMG_SIZE + (3,))

# Tampilkan ringkasan arsitektur model
model.summary()

Membangun model Keras dengan metode Subclassing...




c:\Users\Elan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'batik_lite_model_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "batik_lite_model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Kompilasi model dengan optimizer, loss function, dan metrik
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Mulai proses pelatihan
print(f"\nMemulai pelatihan model untuk {EPOCHS} epoch...")
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCHS
)


Memulai pelatihan model untuk 200 epoch...
Epoch 1/200



34/34 ━━━━━━━━━━━━━━━━━━━━ 38s 751ms/step - accuracy: 0.4297 - loss: 1.5770 - val_accuracy: 0.6519 - val_loss: 1.1725
Epoch 2/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 11s 325ms/step - accuracy: 0.6585 - loss: 1.0848 - val_accuracy: 0.6741 - val_loss: 1.0023
Epoch 3/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 174ms/step - accuracy: 0.6966 - loss: 0.9365 - val_accuracy: 0.7000 - val_loss: 0.9192
Epoch 4/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 165ms/step - accuracy: 0.7332 - loss: 0.8456 - val_accuracy: 0.7074 - val_loss: 0.8683
Epoch 5/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 164ms/step - accuracy: 0.7453 - loss: 0.7824 - val_accuracy: 0.7185 - val_loss: 0.8328
Epoch 6/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 163ms/step - accuracy: 0.7614 - loss: 0.7341 - val_accuracy: 0.7296 - val_loss: 0.8061
Epoch 7/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 163ms/step - accuracy: 0.7701 - loss: 0.6945 - val_accuracy: 0.7370 - val_loss: 0.7851
Epoch 8/200
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 165ms/step - accuracy: 0.7864 - loss: 0.6611 - val_accuracy: 0.75

In [ ]:
print("\nMenghitung dan Menampilkan Metrik Evaluasi pada Validation Set...")

y_true = np.concatenate([y for x, y in validation_dataset], axis=0)

predictions = model.predict(validation_dataset)
y_pred = np.argmax(predictions, axis=1)

print("\n--- Laporan Klasifikasi (Classification Report) ---")
print(classification_report(y_true, y_pred, target_names=class_names))

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')

print(f"\nSkor Akurasi Keseluruhan: {accuracy:.4f} ({accuracy:.2%})")
print(f"Skor Presisi (Weighted): {precision:.4f}")
print(f"Skor Recall (Weighted): {recall:.4f}")

acc = history.history['accuracy']

In [ ]:
# Mengambil data histori pelatihan
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(EPOCHS)

# Membuat plot untuk Akurasi
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

# Membuat plot untuk Loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

# Tampilkan plot
plt.show()

In [ ]:
# Simpan model yang sudah dilatih ke dalam file .h5
NEW_MODEL_PATH = 'batik_lite_model3.h5'
print(f"\nMenyimpan model yang telah dilatih ke '{NEW_MODEL_PATH}'...")

model.save(NEW_MODEL_PATH)

print("Model baru berhasil disimpan!")